In [1]:
import math
import numpy as np
from numpy.random import seed
import random
import pandas as pd

import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

import tensorflow as tf
import tensorflow.keras.utils as Utils
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras import utils as np_utils

2025-06-09 20:23:13.921065: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-09 20:23:13.922709: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-09 20:23:13.969506: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-09 20:23:13.970145: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-09 20:23:14.580074: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# Para reproduzir resultados em todas as execuções
# Seed Everything to reproduce results for future use cases
def seed_everything(seed=42):
    # Seed value for TensorFlow
    tf.random.set_seed(seed)
    
    # Seed value for NumPy
    np.random.seed(seed)
    
    # Seed value for Python's random library
    random.seed(seed)
    
    # Force TensorFlow to use single thread
    # Multiple threads are a potential source of non-reproducible results.
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )

    # Make sure that TensorFlow uses a deterministic operation wherever possible
    tf.compat.v1.set_random_seed(seed)

    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

seed_everything()

In [3]:
#importar funções
from global_functions import *

In [5]:
#Treinar apenas as emoções presentes em todos os datasets (treino e teste)
emotions_labels = {0:'raiva',  1:'medo', 2:'alegria', 3: 'tristeza'}
class_names = ['raiva', 'medo', 'alegria', 'tristeza']

img_width = 224
img_height = 224
img_depth = 3
num_classes = len(class_names)
print('Classes:' + str(num_classes))


Classes:4


In [6]:

#Substituir DATASET por um dos datasets (CK, DEFSS, DEFSS18, FACES, NIMH, MUG)
DATASET = 'NIMH'


## Carregar dados Treino e Teste

In [9]:
X_train = np.load('X_train_'+ DATASET + '.npy')
y_train = np.load('Y_train_' + DATASET + '.npy') 

X_valid = np.load('X_valid_'+ DATASET + '.npy')
y_valid = np.load('Y_valid_' + DATASET + '.npy') 

X_test = np.load('X_test_'+ DATASET + '.npy')
y_test = np.load('Y_test_' + DATASET + '.npy') 

In [10]:
#Criando ImageDataGenerator para augmentation durante o treino
batchSize = 20
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   horizontal_flip = True,
                                   brightness_range=[0.5,1.5]
)

train_datagen.fit(X_train)

valid_datagen = ImageDataGenerator(rescale=1.0/255,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   horizontal_flip = True,
                                   brightness_range=[0.5,1.5]
)

valid_datagen.fit(X_valid)

train_generator = train_datagen.flow(X_train, y_train, batch_size = batchSize)
valid_generator = train_datagen.flow(X_valid, y_valid, batch_size = batchSize)

## Calcular pesos (desbalanceamento classes)

In [11]:
from sklearn.utils import class_weight
classes_train = np.argmax(y_train, axis=1)
classes = np.unique(classes_train)
weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(classes_train), y=classes_train)
weights = np.round(weights,6)
class_weights = {k: v for k, v in zip(classes, weights)}
print('Class weights:', class_weights)

Class weights: {0: 1.060345, 1: 0.904412, 2: 1.025, 3: 1.025}


## Criar modelos e treinar com classes adicionadas

### DenseNet121 - M1

In [12]:
 model = createModel('DenseNet121', img_height, img_width, modelType='m1', num_classes=4)

Model: "model"
_____________________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  Trainable  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            N          
                                                                                                             
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             N          
 g2D)                                                                                                        
                                                                                                             
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      N          
                                                                                                         

 rmalization)                                                       ']                                       
                                                                                                             
 conv2_block3_1_relu (Activ  (None, 56, 56, 128)          0         ['conv2_block3_1_bn[0][0]']   N          
 ation)                                                                                                      
                                                                                                             
 conv2_block3_2_conv (Conv2  (None, 56, 56, 32)           36864     ['conv2_block3_1_relu[0][0]   N          
 D)                                                                 ']                                       
                                                                                                             
 conv2_block3_concat (Conca  (None, 56, 56, 160)          0         ['conv2_block2_concat[0][0]   N          
 tenate)  

 D)                                                                 ']                                       
                                                                                                             
 conv2_block6_concat (Conca  (None, 56, 56, 256)          0         ['conv2_block5_concat[0][0]   N          
 tenate)                                                            ',                                       
                                                                     'conv2_block6_2_conv[0][0]              
                                                                    ']                                       
                                                                                                             
 pool2_bn (BatchNormalizati  (None, 56, 56, 256)          1024      ['conv2_block6_concat[0][0]   N          
 on)                                                                ']                                       
          

 ation)                                                                                                      
                                                                                                             
 conv3_block3_2_conv (Conv2  (None, 28, 28, 32)           36864     ['conv3_block3_1_relu[0][0]   N          
 D)                                                                 ']                                       
                                                                                                             
 conv3_block3_concat (Conca  (None, 28, 28, 224)          0         ['conv3_block2_concat[0][0]   N          
 tenate)                                                            ',                                       
                                                                     'conv3_block3_2_conv[0][0]              
                                                                    ']                                       
          

 tenate)                                                            ',                                       
                                                                     'conv3_block6_2_conv[0][0]              
                                                                    ']                                       
                                                                                                             
 conv3_block7_0_bn (BatchNo  (None, 28, 28, 320)          1280      ['conv3_block6_concat[0][0]   N          
 rmalization)                                                       ']                                       
                                                                                                             
 conv3_block7_0_relu (Activ  (None, 28, 28, 320)          0         ['conv3_block7_0_bn[0][0]']   N          
 ation)                                                                                                      
          

...

 conv5_block2_2_conv (Conv2  (None, 7, 7, 32)             36864     ['conv5_block2_1_relu[0][0]   N          
 D)                                                                 ']                                       
                                                                                                             
 conv5_block2_concat (Conca  (None, 7, 7, 576)            0         ['conv5_block1_concat[0][0]   N          
 tenate)                                                            ',                                       
                                                                     'conv5_block2_2_conv[0][0]              
                                                                    ']                                       
                                                                                                             
 conv5_block3_0_bn (BatchNo  (None, 7, 7, 576)            2304      ['conv5_block2_concat[0][0]   N          
 rmalizati

                                                                    ']                                       
                                                                                                             
 conv5_block6_0_bn (BatchNo  (None, 7, 7, 672)            2688      ['conv5_block5_concat[0][0]   N          
 rmalization)                                                       ']                                       
                                                                                                             
 conv5_block6_0_relu (Activ  (None, 7, 7, 672)            0         ['conv5_block6_0_bn[0][0]']   N          
 ation)                                                                                                      
                                                                                                             
 conv5_block6_1_conv (Conv2  (None, 7, 7, 128)            86016     ['conv5_block6_0_relu[0][0]   N          
 D)       

 ation)                                                                                                      
                                                                                                             
 conv5_block9_1_conv (Conv2  (None, 7, 7, 128)            98304     ['conv5_block9_0_relu[0][0]   N          
 D)                                                                 ']                                       
                                                                                                             
 conv5_block9_1_bn (BatchNo  (None, 7, 7, 128)            512       ['conv5_block9_1_conv[0][0]   N          
 rmalization)                                                       ']                                       
                                                                                                             
 conv5_block9_1_relu (Activ  (None, 7, 7, 128)            0         ['conv5_block9_1_bn[0][0]']   N          
 ation)   

 ormalization)                                                      ]']                                      
                                                                                                             
 conv5_block12_1_relu (Acti  (None, 7, 7, 128)            0         ['conv5_block12_1_bn[0][0]'   N          
 vation)                                                            ]                                        
                                                                                                             
 conv5_block12_2_conv (Conv  (None, 7, 7, 32)             36864     ['conv5_block12_1_relu[0][0   N          
 2D)                                                                ]']                                      
                                                                                                             
 conv5_block12_concat (Conc  (None, 7, 7, 896)            0         ['conv5_block11_concat[0][0   N          
 atenate) 

 2D)                                                                ]']                                      
                                                                                                             
 conv5_block15_concat (Conc  (None, 7, 7, 992)            0         ['conv5_block14_concat[0][0   Y          
 atenate)                                                           ]',                                      
                                                                     'conv5_block15_2_conv[0][0              
                                                                    ]']                                      
                                                                                                             
 conv5_block16_0_bn (BatchN  (None, 7, 7, 992)            3968      ['conv5_block15_concat[0][0   N          
 ormalization)                                                      ]']                                      
          

In [ ]:
 modelTrained = compileAndTrain('DenseNet121', 'm1', DATASET, model, train_generator, valid_generator, X_train, class_weights) 

Epoch 1/30
6/6 [==============================] - 20s 3s/step - loss: 1.6024 - accuracy: 0.2439 - val_loss: 1.3523 - val_accuracy: 0.3902 - lr: 1.0000e-04
Epoch 2/30
6/6 [==============================] - 16s 3s/step - loss: 1.3351 - accuracy: 0.3333 - val_loss: 1.2634 - val_accuracy: 0.3415 - lr: 1.0000e-04
Epoch 3/30
6/6 [==============================] - 17s 3s/step - loss: 1.3374 - accuracy: 0.3984 - val_loss: 1.2301 - val_accuracy: 0.4146 - lr: 1.0000e-04
Epoch 4/30
6/6 [==============================] - 18s 3s/step - loss: 1.3334 - accuracy: 0.3740 - val_loss: 1.1862 - val_accuracy: 0.4878 - lr: 1.0000e-04
Epoch 5/30
6/6 [==============================] - 18s 3s/step - loss: 1.1893 - accuracy: 0.4634 - val_loss: 1.0783 - val_accuracy: 0.6667 - lr: 1.0000e-04
Epoch 6/30
6/6 [==============================] - 18s 3s/step - loss: 1.1553 - accuracy: 0.5285 - val_loss: 1.0057 - val_accuracy: 0.7398 - lr: 1.0000e-04
Epoch 7/30
6/6 [==============================] - 18s 3s/step - loss: 

In [ ]:
yhat = test_model('DenseNet121', 'm1', DATASET, modelTrained, X_test, y_test, emotions_labels)

### DenseNet121 - M2

In [ ]:
model = createModel('DenseNet121', img_height, img_width, modelType='m2', num_classes=4)

In [ ]:
modelTrained = compileAndTrain('DenseNet121', 'm2', DATASET, model, train_generator, valid_generator, 
                                X_train, class_weights) 

In [ ]:
yhat = test_model('DenseNet121', 'm2', DATASET, modelTrained, X_test, y_test, emotions_labels)

### MobileNetV2 - M1

In [ ]:
model = createModel('MobileNetV2', img_height, img_width, modelType='m1', num_classes=4)

In [ ]:
 modelTrained = compileAndTrain('MobileNetV2', 'm1', DATASET, model, train_generator, valid_generator, 
                                X_train, class_weights) 

In [ ]:
yhat = test_model('MobileNetV2', 'm1', DATASET, modelTrained, X_test, y_test, emotions_labels)

### DenseNet201 - M1

In [ ]:
model = createModel('DenseNet201', img_height, img_width, modelType='m1', num_classes=4)

In [ ]:
 modelTrained = compileAndTrain('DenseNet201', 'm1', DATASET, model, train_generator, valid_generator, 
                                X_train, class_weights) 

In [ ]:
yhat = test_model('DenseNet201', 'm1', DATASET, modelTrained, X_test, y_test, emotions_labels)

### DenseNet201 - M2

In [ ]:
model = createModel('DenseNet201', img_height, img_width, modelType='m2', num_classes=4)

In [ ]:
 modelTrained = compileAndTrain('DenseNet201', 'm2', DATASET, model, train_generator, valid_generator, 
                                X_train, class_weights) 

In [ ]:
yhat = test_model('DenseNet201', 'm2', DATASET, modelTrained, X_test, y_test, emotions_labels)